In [ ]:
import os
import sys
os.environ["TRANSFORMERS_CACHE"] = "/workspace/cache/"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
from torch.nn import DataParallel
from utils.prompter import Prompter
from time import time
from time import perf_counter
from peft import PeftModel

from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig, BitsAndBytesConfig

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-30b-hf")

In [ ]:
model = LlamaForCausalLM.from_pretrained("decapoda-research/llama-30b-hf",
       load_in_8bit=True,
        device_map='auto',
        torch_dtype=torch.bfloat16,
    )

model = PeftModel.from_pretrained(
            model,
            "./model/checkpoint-300",
            torch_dtype=torch.float16)

In [ ]:
### model.eval()
if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

In [ ]:

prompter = Prompter("sum")

In [ ]:
instruction = "Résume ce texte issue d'un cours de droit en conservant les dates, les abréviations et les principes importants."
input="""Cet élément est également appelé l’élément moral de la faute puisqu’il permettait de moraliser les comportements. En effet on ne peut sanctionner que les personnes aptes à comprendre la portée de leur acte et donc de les éviter. Plus exactement, cette conception de la faute empêchait de retenir la responsabilité de deux catégories de personnes privées de discernement. Tout d’abord les majeurs atteinte d’un trouble mental qui les empêche de discerner les conséquences de leurs actes mais surtout les enfants en bas âge, privés de discernement. On estimait que la faculté de discernement était atteinte vers l’âge de 7 ans et qu’en-deçà ne discernait pas les conséquences de ses actes."""
prompt = prompter.generate_prompt(instruction, input)

In [ ]:
inputs = tokenizer("My name is ", return_tensors="pt")
input_ids = inputs.input_ids.to("cuda")

In [ ]:
generation_config = GenerationConfig(
    temperature=0,
    top_p=0.75,
    use_cache=False,
    do_sample=True
)
now = time()
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        #generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=100,
    )
duration = time() - now
try:
    s = generation_output.sequences[0]
except:
    s = generation_output[0]

tks = (s.shape[0] - input_ids.shape[1])/duration
print(f"{tks} tokens/s")
print(f"{1/tks} tokens/s")
output = tokenizer.decode(s)
#print(output)
#res = prompter.get_response(output)
print(res)